# San Francisco Rental Prices Dashboard

In this notebook, you will compile the visualizations from the previous analysis into functions that can be used for a Panel dashboard.

In [130]:
# imports
import panel as pn
pn.extension('plotly')
import plotly.express as px
import pandas as pd
import hvplot.pandas
import matplotlib.pyplot as plt
import os
from pathlib import Path
from dotenv import load_dotenv

In [131]:
# Read the Mapbox API key
load_dotenv()
mapbox_token = os.getenv("MAPBOX_API_KEY")
print(type(mapbox_token))

<class 'str'>


# Import Data

In [132]:
# Import the CSVs to Pandas DataFrames
file_path = Path("Data/sfo_neighborhoods_census_data.csv")
sfo_data = pd.read_csv(file_path, index_col="year")

file_path = Path("Data/neighborhoods_coordinates.csv")
sf_neighborhood_locations = pd.read_csv(file_path)
sf_neighborhood_locations.sort_values("Neighborhood",ascending=True,inplace=True)


- - -

## Panel Visualizations

In this section, you will copy the code for each plot type from your analysis notebook and place it into separate functions that Panel can use to create panes for the dashboard. 

These functions will convert the plot object to a Panel pane.

Be sure to include any DataFrame transformation/manipulation code required along with the plotting code.

Return a Panel pane object from each function that can be used to build the dashboard.

Note: Remove any `.show()` lines from the code. We want to return the plots instead of showing them. The Panel dashboard will then display the plots.

In [133]:
#Define Panel Visualization Functions
def housing_units_per_year(sf_mean_data):
    """Housing Units Per Year."""
    return(
        px.line(
            sf_mean_data,
            x="year",
            y="housing_units",
            title="Housing Units per Year")
    )

def average_sales_price(sf_mean_data):
    """Average Sales Price Per Year."""    
    return(
        px.line(
            sf_mean_data,
            x="year",
            y="sale_price_sqr_foot",
            title="Average Sales Price/SqFt per Year")
    )

def average_price_by_neighborhood(sf_neighbor_data):
    """Average Prices by Neighborhood."""
    return(
        px.bar(
            sf_neighbor_data,
            x="neighborhood",
            y="sale_price_sqr_foot",
            title="Average Prices by Neighborhood"
            )
    )

def top_most_expensive_neighborhoods(sf_top10_data):
    """Top 10 Most Expensive Neighborhoods."""
    return(
        px.bar(
            sf_top10_data,
            x="neighborhood",
            y="sale_price_sqr_foot",
            title="Top 10 SF Neighborhoods by Sales Price/SqFt"
            )
    )

def parallel_coordinates(sf_top10_data):
    """Parallel Coordinates Plot."""
    return(
        px.parallel_coordinates(
            sf_top10_data,
            color="sale_price_sqr_foot"
        )
    )

def parallel_categories(sf_top10_data):
    """Parallel Categories Plot."""
    return(
        px.parallel_categories(
            sf_top10_data,
            dimensions=["neighborhood","sale_price_sqr_foot", "housing_units","gross_rent"],
            color="sale_price_sqr_foot",
            color_continuous_scale=px.colors.sequential.Inferno
        )
    )
def neighborhood_map(sf_full_neighbor_data):
    """Neighborhood Map"""
    return(
        px.scatter_mapbox(
            sf_full_neighbor_data,
            lat="Lat",
            lon="Lon",
            size="sale_price_sqr_foot",
            color="gross_rent",
            zoom=10
        )
    )


## Panel Dashboard

In this section, you will combine all of the plots into a single dashboard view using Panel. Be creative with your dashboard design!

In [142]:
sf_mean_data = sfo_data.groupby("year").mean().reset_index()
sf_neighbor_data = sfo_data.groupby(['neighborhood']).mean().sort_values("sale_price_sqr_foot",ascending=True).reset_index()
sf_top10_data = sf_neighbor_data.nlargest(10,columns=["sale_price_sqr_foot"])
sf_full_neighbor_data = pd.concat([sf_neighbor_data,sf_neighborhood_locations],axis='columns',join='inner')
sf_full_neighbor_data = sf_full_neighbor_data.drop(columns=["Neighborhood"])

row_of_avgs = pn.Row(
    housing_units_per_year(sf_mean_data),
    average_sales_price(sf_mean_data),
    average_price_by_neighborhood(sf_neighbor_data)
)

row_of_top10 = pn.Row(
    top_most_expensive_neighborhoods(sf_top10_data),
    parallel_coordinates(sf_top10_data),
    parallel_categories(sf_top10_data)
    )

row_of_maps = pn.Row(neighborhood_map(sf_full_neighbor_data))

sf_dashboard = pn.Tabs(
    ("Yearly Averages",row_of_avgs),
    ("Ten Most Expensive",row_of_top10),
    ("City Neighborhoods Map",row_of_maps)
)
sf_dashboard

Tabs
    [0] Row
        [0] Plotly(Figure)
        [1] Plotly(Figure)
        [2] Plotly(Figure)
    [1] Row
        [0] Plotly(Figure)
        [1] Plotly(Figure)
        [2] Plotly(Figure)
    [2] Row
        [0] Plotly(Figure)

## Serve the Panel Dashboard

In [143]:
sf_dashboard.servable()

Tabs
    [0] Row
        [0] Plotly(Figure, viewport={'xaxis.range': [2010, ...})
        [1] Plotly(Figure, viewport={'xaxis.range': [2010, ...})
        [2] Plotly(Figure, viewport={'xaxis.range': [-0.5, ...})
    [1] Row
        [0] Plotly(Figure, viewport={'xaxis.range': [-0.5, ...})
        [1] Plotly(Figure)
        [2] Plotly(Figure)
    [2] Row
        [0] Plotly(Figure)